In [2]:
import re
import json

file = open(
    "./notebooks/14-23年07049心理咨询与技术真题及答案【艺林整理】/14-23年07049心理咨询与技术真题及答案【艺林整理】.txt",
    "r",
)

content = "\n".join(
    line for line in file.read().splitlines() if line and "内部资料，保密承诺" not in line
)

pattern = r"(.+?高等教育自学考试.+?)(?=\n?\d+ 年 \d+ 月高等教育自学考试|$)"
matches = re.findall(pattern, content, re.DOTALL)
cleaned_matches = [match.strip() for match in matches]

parsed = {
    "categories": [],
    "questions": [],
    "answers": [],
    "explains": [],
}


for i, match in enumerate(cleaned_matches):
    match = re.split(r".*答案.*", match.replace("\n（）", "（）"))

    raw_questions = match[0].strip()
    raw_answers = (match[1] if len(match) > 1 else "").strip()

    answers = [
        _a.strip()
        for atype in re.split(r"(?=\n[一二三四五六七八九十])", raw_answers)
        for _a in re.split(r'(?=\n\d+\.)', '\n'.join(atype.strip().split('\n')[1:]))
    ]


    bank = "心理咨询原理与技术(课程代码 07049)"
    category = raw_questions.splitlines()[0]

    if category not in parsed["categories"]:
        parsed["categories"].append(category)

    for qtype in re.split(r"(?=\n[一二三四五六七八九十])", raw_questions):
        split_qtype = qtype.strip().split('\n')

        _qtype = re.findall(r"、(.*)|$", split_qtype[0]).pop(0)

        for _q in re.split(r'(?=\n\d+\.)', '\n'.join(split_qtype[1:])):
            parsed["questions"].append({
                "id": _q.split('.')[0].strip(),
                "bank": bank.strip(),
                "category": category.strip(),
                "type": _qtype.strip(),
                "content": _q.strip(),
            })

    for atype in re.split(r"(?=\n[一二三四五六七八九十])", raw_answers):
        split_atype = atype.strip().split('\n')

        _atype = re.findall(r"、(.*)|$", split_atype[0]).pop(0)

        for _a in re.split(r'(?=\n\d+\.)', '\n'.join(split_atype[1:])):
            parsed['answers'].append({
                "id": _a.split('.')[0].strip(),
                "bank": bank.strip(),
                "category": category.strip(),
                "type": _atype.strip(),
                "content": _a.strip()
            })


with open(
    f"./src/assets/心理咨询原理与技术(课程代码 07049).json",
    "w",
) as f:
    json.dump(parsed, f, ensure_ascii=False, indent=2)
